# ETL Reporte para OBEX

* Extraer la data de los JSON
* Transformar en una tabla con los campos requeridos
* Guardar en formato CSV

In [46]:
import boto3
import json
import pandas as pd
from tqdm.notebook import trange, tqdm

In [24]:
bucketname = 'bi-obex-study-json'
test_file = '77559056-4.json'

In [26]:
client = boto3.client('s3')

In [ ]:
#Create a file object using the bucket and object key. 
fileobj = client.get_object(
    Bucket=bucketname,
    Key=test_file
    ) 
# open the file object and read it into the variable filedata. 
filedata = fileobj['Body'].read()

# file data will be a binary stream.  We have to decode it 
test_json = json.loads(filedata)

#print(json.dumps(test_json, indent=4, sort_keys=True))

In [263]:

paginator = client.get_paginator('list_objects_v2')
pages = paginator.paginate(Bucket=bucketname)

data = []

j=0
for page in tqdm(pages):
    for obj in tqdm(page['Contents']):
        
        if j < 10000000:
            # Read data
            fileobj = client.get_object(Bucket=bucketname,
                                        Key=obj['Key']
                                       ) 
            filedata = fileobj['Body'].read().decode('utf-8') 
            obj_json = json.loads(filedata)

            # Parse and create data row
            item = dict()
            item['rut'] = obj_json['data']['estado']['rut']
            item['razon_social'] = obj_json['data']['estado']['razonSocial']
            #item['fecha_actuacion'] = obj_json['data']['estado']['fecha']
            item['fecha_actuacion'] = obj_json['data']['estudio']['prestudios'][0]['fecha']
            item['fecha_estudio'] = obj_json['data']['estudio']['fechaEstudio']
            item['socios'] = obj_json['data']['estudio']['prestudios'][-1]['accionistas']['valor']
            item['pre_aprobado'] = obj_json['data']['estudio']['preAprobado']
            item['tipo_firma'] = obj_json['data']['estudio']['prestudios'][-1]['administracion']['valor']['comoAdministran']
            item['apoderados'] = obj_json['data']['estudio']['prestudios'][-1]['administracion']['valor']['administradores']
            data.append(item)
        j+=1
        
print(j)



5653


In [264]:
df = pd.DataFrame(data)

## Transformations

In [265]:
df.head(10)

,rut,razon_social,fecha_actuacion,fecha_estudio,socios,pre_aprobado,tipo_firma,apoderados
0,775590564,Minimarket Makarena Krause E.I.R.L.,2022-04-16,2022-05-02 19:15:10.509874,"[{'nombre': 'MAKARENA DENISS KRAUSE VEGA', 'ru...",False,individualmente,"[{'nombre': 'MAKARENA DENISS KRAUSE VEGA', 'ru..."
1,775590572,Adonnay Beauty SpA,2022-04-16,2022-05-02 19:26:48.053295,"[{'nombre': 'FELIPE IGNACIO ILABACA LEAL', 'ru...",True,individualmente,"[{'rut': '18.174.910-5', 'nombre': 'felipe ign..."
2,775590580,BARROS DURAN SPA,2022-04-16,2022-05-02 19:29:01.014415,"[{'nombre': 'DHORTTY IRIS BARROS DURÁN', 'rut'...",True,individualmente,"[{'rut': '12.372.700-2', 'nombre': 'dhortty ir..."
3,775590599,Prevención y cumplimiento Integral SpA,2022-04-16,2022-05-02 18:49:20.426186,[{'nombre': 'VISNA DEL PILAR ORELLANA MARTÍNEZ...,True,individualmente,"[{'rut': '11.930.118-1', 'nombre': 'visna del ..."
4,775590610,Consultora Óscar Pacheco Parra E.I.R.L.,2022-04-16,2022-05-02 19:12:03.149842,"[{'nombre': 'OSCAR BENEDICTO PACHECO PARRA', '...",True,individualmente,"[{'nombre': 'OSCAR BENEDICTO PACHECO PARRA', '..."
5,775590629,Consultoría y producción de eventos Coach-Arte...,2022-04-16,2022-05-02 17:46:23.717461,"[{'nombre': 'MARÍA LORENA MARTÍNEZ SALAMANCA',...",True,indistintamente,"[{'rut': '17.314.903-4', 'nombre': 'maria lore..."
6,775590637,beethoven spa,2022-04-16,2022-05-02 18:54:53.329299,"[{'nombre': 'FRANCISCA GROSS DEMPSTER', 'rut':...",True,individualmente,"[{'rut': '9.290.930-1', 'nombre': 'francisca g..."
7,775590645,bach spa,2022-04-16,2022-05-02 17:46:15.633451,"[{'nombre': 'FRANCISCA GROSS DEMPSTER', 'rut':...",True,individualmente,"[{'rut': '9.290.930-1', 'nombre': 'francisca g..."
8,775590653,mozart spa,2022-04-16,2022-05-02 19:45:17.758505,"[{'nombre': 'FRANCISCA GROSS DEMPSTER', 'rut':...",False,individualmente,"[{'rut': '9.290.930-1', 'nombre': 'francisca g..."
9,775590661,SANTO SANTINO UNO SPA,2022-04-16,2022-05-02 19:13:27.253546,"[{'nombre': 'MARTIN ADRIAN FERNANDEZ', 'rut': ...",True,individualmente,"[{'rut': '22.814.643-9', 'nombre': 'martin adr..."


Fechas

In [266]:
#df['fecha_actuacion_2'] = pd.to_datetime(df['fecha_actuacion'], format='%Y%m%d%H%M%S%f')
df['fecha_actuacion_2'] = pd.to_datetime(df['fecha_actuacion'], format='%Y-%m-%d')

In [267]:
df.head(2)

,rut,razon_social,fecha_actuacion,fecha_estudio,socios,pre_aprobado,tipo_firma,apoderados,fecha_actuacion_2
0,775590564,Minimarket Makarena Krause E.I.R.L.,2022-04-16,2022-05-02 19:15:10.509874,"[{'nombre': 'MAKARENA DENISS KRAUSE VEGA', 'ru...",False,individualmente,"[{'nombre': 'MAKARENA DENISS KRAUSE VEGA', 'ru...",2022-04-16
1,775590572,Adonnay Beauty SpA,2022-04-16,2022-05-02 19:26:48.053295,"[{'nombre': 'FELIPE IGNACIO ILABACA LEAL', 'ru...",True,individualmente,"[{'rut': '18.174.910-5', 'nombre': 'felipe ign...",2022-04-16


Expandir Socios

In [268]:
#df.loc[0, 'socios']

In [269]:
df['n_socios'] = df['socios'].apply(lambda x: len(x))
df['n_apoderados'] = df['apoderados'].apply(lambda x: len(x))

In [281]:
def expand_list(series, field):
    list_df = pd.DataFrame(series.tolist())
    for col in list_df.columns:
    #for col in [0]:
        list_df[field + '_' + series.name + '_' + str(col + 1)] = \
            list_df[col].apply(lambda x: x.get(field) if x is not None else None)
        del list_df[col]
    return list_df

def clean_ruts(series):
    series = series.str.replace('\.', '')
    series = series.str.replace('-', '')
    
    return series

In [282]:
df_socios = expand_list(df['socios'], 'rut')
df_apoderados = expand_list(df['apoderados'], 'rut')

for col in df_socios:
    df_socios[col] = clean_ruts(df_socios[col])
for col in df_apoderados:
    df_apoderados[col] = clean_ruts(df_apoderados[col])
    

In [284]:
df1 = pd.concat([df, df_socios, df_apoderados], axis=1)

In [286]:
df1.head(2)

,rut,razon_social,fecha_actuacion,fecha_estudio,socios,pre_aprobado,tipo_firma,apoderados,fecha_actuacion_2,n_socios,...,rut_socios_38,rut_socios_39,rut_socios_40,rut_apoderados_1,rut_apoderados_2,rut_apoderados_3,rut_apoderados_4,rut_apoderados_5,rut_apoderados_6,rut_apoderados_7
0,775590564,Minimarket Makarena Krause E.I.R.L.,2022-04-16,2022-05-02 19:15:10.509874,"[{'nombre': 'MAKARENA DENISS KRAUSE VEGA', 'ru...",False,individualmente,"[{'nombre': 'MAKARENA DENISS KRAUSE VEGA', 'ru...",2022-04-16,1,...,None,None,None,186527518,None,None,None,None,None,None
1,775590572,Adonnay Beauty SpA,2022-04-16,2022-05-02 19:26:48.053295,"[{'nombre': 'FELIPE IGNACIO ILABACA LEAL', 'ru...",True,individualmente,"[{'rut': '18.174.910-5', 'nombre': 'felipe ign...",2022-04-16,1,...,None,None,None,181749105,None,None,None,None,None,None


Organizar columnas

In [287]:
del df1['fecha_actuacion']
df1 = df1.rename(columns={'fecha_actuacion_2' : 'fecha_actuacion'})

In [288]:
del df1['socios']
del df1['apoderados']

Reorder Columns

In [289]:
max_socios = df1['n_socios'].max()
max_apoderados = df1['n_apoderados'].max()

In [290]:
col_socios = ['rut_socios_' + str(i) for i in range(1, max_socios + 1)]
col_apoderados = ['rut_apoderados_' + str(i) for i in range(1, max_apoderados + 1)]

In [291]:
df1.columns

Index(['rut', 'razon_social', 'fecha_estudio', 'pre_aprobado', 'tipo_firma',
       'fecha_actuacion', 'n_socios', 'n_apoderados', 'rut_socios_1',
       'rut_socios_2', 'rut_socios_3', 'rut_socios_4', 'rut_socios_5',
       'rut_socios_6', 'rut_socios_7', 'rut_socios_8', 'rut_socios_9',
       'rut_socios_10', 'rut_socios_11', 'rut_socios_12', 'rut_socios_13',
       'rut_socios_14', 'rut_socios_15', 'rut_socios_16', 'rut_socios_17',
       'rut_socios_18', 'rut_socios_19', 'rut_socios_20', 'rut_socios_21',
       'rut_socios_22', 'rut_socios_23', 'rut_socios_24', 'rut_socios_25',
       'rut_socios_26', 'rut_socios_27', 'rut_socios_28', 'rut_socios_29',
       'rut_socios_30', 'rut_socios_31', 'rut_socios_32', 'rut_socios_33',
       'rut_socios_34', 'rut_socios_35', 'rut_socios_36', 'rut_socios_37',
       'rut_socios_38', 'rut_socios_39', 'rut_socios_40', 'rut_apoderados_1',
       'rut_apoderados_2', 'rut_apoderados_3', 'rut_apoderados_4',
       'rut_apoderados_5', 'rut_apoderado

In [292]:
df1.shape

(5653, 55)

In [293]:
cols = ['rut', 'razon_social', 'tipo_firma', 'fecha_actuacion', 'fecha_estudio',
              'pre_aprobado'] + ['n_socios'] + ['n_apoderados'] + col_socios + col_apoderados
#cols

In [294]:
df1 = df1.loc[:, cols]

## EDA

In [295]:
df1['tipo_firma'].value_counts()

individualmente              4445
indistintamente               657
conjuntamente                 258
separadamente                 247
no determinado                 35
un representante no socio      11
Name: tipo_firma, dtype: int64

In [296]:
df1['pre_aprobado'].value_counts()

True     4849
False     804
Name: pre_aprobado, dtype: int64

In [297]:
df1['n_socios'].value_counts()

1     3903
2     1482
3      181
4       61
5       14
17       3
7        2
6        2
15       1
10       1
9        1
40       1
8        1
Name: n_socios, dtype: int64

In [298]:
df1['n_apoderados'].value_counts()

1    4470
2     997
3     131
0      27
4      22
5       3
6       2
7       1
Name: n_apoderados, dtype: int64

In [299]:
vol_dia = df1.groupby(pd.Grouper(key='fecha_actuacion', freq='d'))['rut'].count()
vol_dia

fecha_actuacion
2022-03-16      1
2022-03-17      1
2022-03-18      0
2022-03-19      0
2022-03-20      0
2022-03-21      0
2022-03-22      1
2022-03-23      1
2022-03-24      1
2022-03-25      0
2022-03-26      0
2022-03-27      0
2022-03-28      0
2022-03-29      1
2022-03-30      1
2022-03-31      2
2022-04-01      0
2022-04-02      0
2022-04-03      4
2022-04-04      2
2022-04-05      1
2022-04-06      4
2022-04-07      5
2022-04-08      3
2022-04-09      1
2022-04-10      2
2022-04-11      8
2022-04-12      7
2022-04-13     13
2022-04-14     12
2022-04-15      7
2022-04-16     68
2022-04-17     75
2022-04-18    578
2022-04-19    630
2022-04-20    521
2022-04-21    419
2022-04-22    447
2022-04-23      0
2022-04-24     62
2022-04-25    614
2022-04-26    616
2022-04-27    538
2022-04-28    483
2022-04-29    436
2022-04-30     88
Freq: D, Name: rut, dtype: int64

## Output

In [300]:
df1['razon_social'] = df1['razon_social'].str.normalize('NFKD').str.title() \
                    .str.encode('ascii', errors='ignore').str.decode('utf-8')

In [301]:
#df1

In [302]:
df1.to_csv('../reports/100. Obex intelligence/corporate_intelligence_DEMO-202205.csv', 
           index=False,
           encoding='utf-8',
           quoting=2)